In [2]:
import matplotlib.pyplot as plt

import torch
from torch import nn
from torch import optim

import torch.nn.functional as F
from torch.autograd import Variable
from torchvision import datasets,transforms,models


In [4]:
data_dir=r'cat_dog\Cat_Dog_data\train'
data_dir_test=r'cat_dog\Cat_Dog_data\test'

train_transform = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])])


test_transform=transforms.Compose([transforms.Resize(255),
                              transforms.CenterCrop(224),
                             transforms.ToTensor(),
                                 transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])])

In [6]:
train_dataset = datasets.ImageFolder(data_dir, transform=train_transform)
test_dataset = datasets.ImageFolder(data_dir_test, transform=test_transform)

trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
testloader=torch.utils.data.DataLoader(test_dataset, batch_size=32)

In [7]:
model=models.densenet121(pretrained=True)

Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to C:\Users\Deep_Learning/.cache\torch\hub\checkpoints\densenet121-a639ec97.pth


  0%|          | 0.00/30.8M [00:00<?, ?B/s]

In [9]:
model

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [10]:
# freeze our feature parameters

for param in model.parameters():
    param.requires_grad=False

In [14]:
!pip install collections-extended
from collections import OrderedDict

In [15]:
classifier=nn.Sequential(OrderedDict([
    ('fc1',nn.Linear(1024,500)),
    ('relu',nn.ReLU()),
    ('fc2',nn.Linear(500,2)),
    ('output',nn.LogSoftmax(dim=1))
]))

In [16]:
classifier

Sequential(
  (fc1): Linear(in_features=1024, out_features=500, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=500, out_features=2, bias=True)
  (output): LogSoftmax(dim=1)
)

In [17]:
model.classifier=classifier

In [26]:
model

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [19]:
import time

In [24]:
device=torch.device('cuda' if torch.cuda.is_available() else "cpu")

In [28]:
criterion=nn.NLLLoss()
optimizer=optim.Adam(model.classifier.parameters(),lr=0.003)
model.to(device)

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [33]:
epochs=1
steps=0
running_loss=0
print_every=5
train_losses = []
test_losses = []
for e in range(epochs):
    for images,labels in trainloader:
        steps+=1
        
        images,labels=images.to(device),labels.to(device)
        optimizer.zero_grad()
        logps=model(images)
        
        loss=criterion(logps,labels)
        loss.backward()
        optimizer.step()
        
        running_loss+=loss.item()
        
    else:
        with torch.no_grad():
            model.eval()
            test_loss=0
            accuracy=0
            
            for images,labels in testloader:
                images,labels=images.to(device),labels.to(device)
                logps=model(images)
                
                loss=criterion(logps,labels)
                
                test_loss+=loss.item()
                
                ps=torch.exp(logps)
                top_ps,top_class=ps.topk(1,dim=1)
                equality=top_class==labels.view(*top_class.shape)
                
                accuracy+=torch.mean(equality.type(torch.FloatTensor)).item()
        train_losses.append(running_loss / len(trainloader))
        test_losses.append(test_loss / len(testloader))  
        print ('Epochs {}/{}'.format(e+1, epochs),
               'Train Loss: {:.3f}'.format(train_losses[-1]),
               'Test Loss: {:.3f}'.format(test_losses[-1]),
               'Accuracy {:.3f}'.format((accuracy / len(testloader)) * 100 ))
        
        model.train()
                
                

Epochs 1/1 Train Loss: 0.119 Test Loss: 0.041 Accuracy 98.259


## Resnet Pretrained


In [25]:
model1=models.resnet50(pretrained=True)

